In [1]:
import jax
import jax.numpy as jnp
import numpy as np

In [2]:
from arraytainers import Arraytainer, Jaxtainer

In [4]:
A = Arraytainer({'a': {'c': [np.array([[1,2],[3,4],[5,6]])]}, 'b': [{'a':np.array([6,5,4,3,2,1])}]})
shape = Arraytainer({'a': {'c': [(1,2,3)]}, 'b': (3,2)}, array_conversions=False)
A.reshape(shape)

Arraytainer({'a': {'c': [(1, 2, 3)]}, 'b': (3, 2)})


ValueError: zero-dimensional arrays cannot be concatenated

In [6]:
A = Arraytainer({'a': np.array([[1,2],[3,4],[5,6]]), 'b': np.array([6,5,4,3,2,1])})
shape = (Arraytainer({'a': (1,2,3), 'b': (3,2)}),)
A.reshape(shape)

Args = (array([[1, 2],
       [3, 4],
       [5, 6]]), array([1, 2, 3]))
Args = (array([6, 5, 4, 3, 2, 1]), array([3, 2]))


Arraytainer({'a': array([[[1, 2, 3],
        [4, 5, 6]]]), 'b': array([[6, 5],
       [4, 3],
       [2, 1]])})

In [5]:
A = Arraytainer([{'a':1, 1:2}, {0:3, 1:4, 'a':{0:5}}, [6, 7, {'a':8, 'b':9}]])
A

Arraytainer([{'a': array(1), 1: array(2)}, {0: array(3), 1: array(4), 'a': {0: array(5)}}, [array(6), array(7), {'a': array(8), 'b': array(9)}]])

In [6]:
B = Jaxtainer([{'a':1, 1:2}, {0:3, 1:4, 'a':{0:5}}, [6, 7, {'a':8, 'b':9}]])
type(B[0]['a'])

int

In [6]:
A = Arraytainer({'a': -1*np.ones((2,2)), 'b': np.ones((2,2))})
B = Arraytainer({'a': np.array([[True, False],[True, False]]), 'b': np.array([True, False])})
C = Arraytainer({'a': 2, 'b': np.array([3,4])})

In [7]:
A[B] = C
A

KeyError: '.'

In [3]:
A = Jaxtainer({'a': jnp.array([1])}, dtype=float)
A[1] = 1
A.first_array

DeviceArray([1.], dtype=float32)

In [4]:
A = Arraytainer({'a':[3, 2], 'b':[1, 2]}, dtype=int)
B = Arraytainer({'b': np.array([3])}, dtype=float)
A.first_array

array(3)

In [5]:
B.reshape(1,B.shape)

Arraytainer({'b': array([[3.]])})

In [6]:
list(B.values())

[array([3.])]

In [8]:
A.size

4

AttributeError: 'Arraytainer' object has no attribute 'max'